In [ ]:
# | default_exp _components.test_dependencies

# Install Test Dependencies

In [ ]:
# | export
import shutil
import tarfile
from os import environ
from pathlib import Path
from typing import *

import jdk
import requests

from fastkafka._components.helpers import in_notebook
from fastkafka._components.logger import get_logger

if in_notebook():
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

In [ ]:
from fastkafka._components.logger import supress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


def check_java(*, potential_jdk_path: Optional[List[Path]] = None) -> bool:
    if potential_jdk_path is None:
        potential_jdk_path = list(Path(environ["HOME"] + "/.jdk").glob("jdk-11*"))

    if potential_jdk_path != []:
        logger.info("Java is already installed.")
        if not shutil.which("java"):
            logger.info("But not exported to PATH, exporting...")
            environ["PATH"] = environ["PATH"] + f":{potential_jdk_path[0]}/bin"
        return True
    return False

In [ ]:
assert check_java()

[INFO] __main__: Java is already installed.
[INFO] __main__: But not exported to PATH, exporting...


In [ ]:
# | export


def _install_java() -> None:
    """Checks if jdk-11 is installed on the machine and installs it if not
    Returns:
       None
    """
    if not check_java():
        logger.info("Installing Java...")
        logger.info(" - installing jdk...")
        jdk_bin_path = jdk.install("11")
        logger.info(f" - jdk path: {jdk_bin_path}")
        environ["PATH"] = environ["PATH"] + f":{jdk_bin_path}/bin"
        logger.info("Java installed.")

In [ ]:
# | notest

_install_java()
assert shutil.which("java")
_install_java()
assert shutil.which("java")

[INFO] __main__: Java is already installed.
[INFO] __main__: Java is already installed.


In [ ]:
# | export

# ToDo: move it somewhere
kafka_version = "3.3.2"
kafka_fname = f"kafka_2.13-{kafka_version}"
kafka_url = f"https://dlcdn.apache.org/kafka/{kafka_version}/{kafka_fname}.tgz"
local_path = Path(environ["HOME"]) / ".local"
tgz_path = local_path / f"{kafka_fname}.tgz"
kafka_path = local_path / f"{kafka_fname}"


def check_kafka(kafka_path: Path = kafka_path) -> bool:
    if (kafka_path / "bin").exists():
        logger.info("Kafka is installed.")
        if not shutil.which("kafka-server-start.sh"):
            logger.info("But not exported to PATH, exporting...")
            environ["PATH"] = environ["PATH"] + f":{kafka_path}/bin"
        return True
    return False

In [ ]:
# | export


def _install_kafka(
    *,
    kafka_url: str = kafka_url,
    local_path: Path = local_path,
    tgz_path: Path = tgz_path,
) -> None:
    """Checks if kafka is installed on the machine and installs it if not
    Returns:
       None
    """
    if not check_kafka():
        logger.info("Installing Kafka...")
        local_path.mkdir(exist_ok=True, parents=True)
        response = requests.get(
            kafka_url,
            stream=True,
        )
        try:
            total = response.raw.length_remaining // 128
        except Exception:
            total = None

        with open(tgz_path, "wb") as f:
            for data in tqdm(response.iter_content(chunk_size=128), total=total):
                f.write(data)

        with tarfile.open(tgz_path) as tar:
            for tarinfo in tar:
                tar.extract(tarinfo, local_path)

        environ["PATH"] = environ["PATH"] + f":{kafka_path}/bin"
        logger.info(f"Kafka installed in {kafka_path}.")

In [ ]:
# | notest

_install_kafka()
assert shutil.which("kafka-server-start.sh")
_install_kafka()
assert shutil.which("kafka-server-start.sh")

[INFO] __main__: Kafka is installed.
[INFO] __main__: But not exported to PATH, exporting...
[INFO] __main__: Kafka is installed.


In [ ]:
# | export


def _install_testing_deps() -> None:
    _install_java()
    _install_kafka()

In [ ]:
_install_testing_deps()
assert shutil.which("java")
assert shutil.which("kafka-server-start.sh")

[INFO] __main__: Java is already installed.
[INFO] __main__: Kafka is installed.
